In [ ]:
congresses = [118, 117, 119] # year 2000-2025
bill_types = ["hr", "s", "hjres", "sjres"] # house of reps, senate, house joint res, senate joint res"

In [ ]:
import pandas as pd
import ast
import os

candidates = pd.DataFrame(columns=["LastName", "Name", "State", "District", "Party", "Term", "ImageUrl"])
unique_candidates = set()
i = 0

for c in congresses:
    for b_t in bill_types:
        file = f"{c}_{b_t}.csv"
        if os.path.exists(vote_table_name):
            print(file)
            df = pd.read_csv("members_df.csv")
            for index, row in df.iterrows():
                if row["Congress"] in congresses:
                    name = row["Name"]
                    last_name = name.split(", ")[0]
                    state = row["State"]
                    unique_key = last_name + state
                    if unique_key in unique_candidates:
                        continue
                    else:
                        unique_candidates.add(unique_key)
                        district = row["District"]
                        party = row["Party"]
                        term = row["Term"]
                        image = row["ImageUrl"]
                        candidates.loc[i] = {"LastName":last_name, "Name":name, "State":state, "District":district, "Party":party, "Term":term, "ImageUrl":image}
                        i += 1

candidates.to_csv(f"candidates.csv", index=False)

In [ ]:
# Iterate through the DataFrame
for index, row in df.iterrows():
    name = row['Name']
    term = row['Term']
    
    # If the member already exists, append the term to their list
    if name in members_dict:
        members_dict[name].append(term)
    else:
        # Otherwise, create a new list with their term
        members_dict[name] = [term]

# Display the result
for name, terms in members_dict.items():
    print(f"Member: {name}")
    for term in terms:
        print(f"  Term: {term}")

In [ ]:
VOTE_TABLE_PATH = "vote_tables/"
VOTE_MAPPING_PATH = "summaries_and_vote_mapping/"

# congresses = [i for i in range(115, 120)] # year 2000-2025
congresses = [118, 117, 119] # year 2000-2025
congresses.reverse()
bill_types = ["hr", "s", "hjres", "sjres"] # house of reps, senate, house joint res, senate joint res"
# bill_types = ["hr"]

In [ ]:
import pandas as pd
import ast
import os

candidates = pd.DataFrame(columns=["Name", "Processed_Name", "State", "Party", "Congresses"])
unique_candidates = {}
i = 0

for c in congresses:
    for b_t in bill_types:
        file = f"{c}_{b_t}.csv"
        vote_table_name = VOTE_TABLE_PATH + file
        if os.path.exists(vote_table_name):
            print(file)
            v_t_df = pd.read_csv(vote_table_name)
            for index, row in v_t_df.iterrows():
                name = row["Name"]
                state = row["State"]
                unique_key = name + state
                if unique_key in unique_candidates:
                    if c not in candidates.loc[unique_candidates[unique_key]]["Congresses"]:
                        candidates.loc[unique_candidates[unique_key]]["Congresses"].append(c)
                else:
                    unique_candidates[unique_key] = i
                    party = row["Party"]
                    processed_name = name.split(" (")[0]
                    unique_key = name + " (" + state  
                    candidates.loc[i] = {"Name":name, "Processed_Name":processed_name, "State":state, "Party":party, "Congresses":[c]}
                    i += 1

candidates.to_csv(f"candidate_profiles/candidates.csv", index=False)

In [ ]:
import pandas as pd
import ast
import os

candidates = pd.DataFrame(columns=["UniqueKey", "Name", "State", "Party", "Scores"])

for c in congresses:
    for b_t in bill_types:
        file = f"{c}_{b_t}.csv"
        vote_table_name = VOTE_TABLE_PATH + file
        vote_mapping_name = VOTE_MAPPING_PATH + file
        if os.path.exists(vote_table_name) and os.path.exists(vote_mapping_name):
            print(file)
            v_t_df = pd.read_csv(vote_table_name)
            v_m_df = pd.read_csv(vote_mapping_name)
            for index, row in v_m_df.iterrows():
                vote_mapping = ast.literal_eval(row["VoteMapping"])
                # print(vote_mapping)
                if len(vote_mapping) == 4: # processed correctly
                    filtered_df = v_t_df[(v_t_df['Bill'] == row["Bill"]) & (v_t_df['DateTime'] == row['VotesDateTime'])]
                    bill = row["Bill"]
                    data_time = row['VotesDateTime']
                    unique_key = bill + ", " + data_time
                    for f_index, f_row in filtered_df.iterrows():
                        name = f_row["Name"]
                        state = f_row["State"]
                        unique_key = name + "_" + state
                        party = f_row["Party"]
                        score_dict = {}
                        if f_row["Vote"] == "Y":
                            if "Yea_Social_Scale" in vote_mapping:
                                score_dict["Yea_Social_Scale"] = vote_mapping["Yea_Social_Scale"]
                            elif "Yea_Social" in vote_mapping:
                                score_dict["Yea_Social_Scale"] = vote_mapping["Yea_Social"]
                            if "Yea_Economic_Scale" in vote_mapping:
                                score_dict["Yea_Economic_Scale"] = vote_mapping["Yea_Economic_Scale"]
                            elif "Yea_Economic" in vote_mapping:
                                score_dict["Yea_Economic_Scale"] = vote_mapping["Yea_Economic"]
                        elif f_row["Vote"] == "N":
                            if "Nay_Social_Scale" in vote_mapping:
                                score_dict["Nay_Social_Scale"] = vote_mapping["Nay_Social_Scale"]
                            elif "Nay_Social" in vote_mapping:
                                score_dict["Nay_Social_Scale"] = vote_mapping["Nay_Social"]
                            if "Nay_Economic_Scale" in vote_mapping:
                                score_dict["Nay_Economic_Scale"] = vote_mapping["Nay_Economic_Scale"]
                            elif "Nay_Economic" in vote_mapping:
                                score_dict["Yea_Economic_Scale"] = vote_mapping["Nay_Economic"]
                            
                        if ((candidates['Name'] == name) & (candidates['State'] == state)).any(): # already present
                            candidates.loc[(candidates['Name'] == name) & (candidates['State'] == state), "Scores"][unique_key] = score_dict
                        else:
                            candidates.loc[len(candidates)] = {"Name":name, "State":state, "Party":party, "Scores":{unique_key:score_dict}}

candidates.to_csv(f"candidate_profiles/candidates.csv", index=False)

In [ ]:
df = pd.read_csv("candidate_profiles/candidates.csv")
for index, row in df.iterrows():
    print(row["Name"], row["Scores"])